## Reframing api_code.ipynb as a set of functions

In [1]:
def get_client(endpoint = None, api_key = None, api_version = None):
    from openai import AzureOpenAI
    import os
    if endpoint is None:
        endpoint = os.environ["AZURE_OPENAI_ENDPOINT"] 
    if api_key is None:
        api_key = os.environ["AZURE_OPENAI_API_KEY"] 
    if api_version is None:
        api_version = os.environ["AZURE_OPENAI_API_VERSION"]

    client = AzureOpenAI( api_key = api_key,  api_version = api_version, azure_endpoint = endpoint)
    return client


In [2]:
def get_prompt(system_prompt = None, user_prompt = None, context = None):
    if system_prompt is None:
        system_prompt = "You are a helpful assistant. For each call, use provided context (Context:) to answer a provided question (Question:) in a concise manner.  Return only the answer."
    if user_prompt is None:
        user_prompt = "Echo whatever is provided as context."
    if context is None:
        context = "Echo"
    return [
    {
        "role": "system",
        "content": system_prompt
    },
    {
        "role": "user",
        "content": f"Question: {user_prompt}. Context: {context}" 
    }
    ]


In [3]:
def call_llm(prompt = None, client = None, deployment = None, temperature = None):
    if prompt is None:
        prompt = get_prompt()
    if client is None:
        client = get_client()
    if deployment is None:
        deployment = os.environ["DEPLOYMENT_NAME"]
    if temperature is None:
        temperature = 0.7
    completion = client.chat.completions.create(
        model = deployment,
        messages = prompt,
        temperature = temperature
    )  
    return completion.choices[0].message.content

In [4]:
def execute(context = None, user_prompt=None, system_prompt= None):
    prompt = get_prompt(context = context, user_prompt = user_prompt, system_prompt = system_prompt)
    client = get_client()
    return call_llm(prompt = prompt, client = client)


In [7]:
import pandas as pd
input_data = pd.read_csv("book_desc.csv", skipinitialspace=True)

print(input_data.shape)

import os
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://oai-test-ss.openai.azure.com/"
os.environ["AZURE_OPENAI_API_KEY"] = ""
os.environ["DEPLOYMENT_NAME"] = "gpt-35-turbo"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-10-21"

user_prompt = "Provide a summary of this book in one sentence."
system_prompt = "You are a helpful library assistant who categorises books or summarises them, or provides recommendations."

input_data["response"] = input_data["description"].apply(execute, user_prompt = user_prompt, system_prompt = system_prompt)

(3, 3)


In [8]:
input_data.head(10)

,title,author,description,response
0,Onyx Storm,Rebecca Yarros,After nearly eighteen months at Basgiath War C...,The book follows Violet Sorrengail as she jour...
1,Beautiful Ugly,Alice Feeney,Author Grady Green is having the worst best da...,"""The Other Mrs. by Mary Kubica"" is a thrilling..."
2,Great Big Beautiful Life,Emily Henry,Alice Scott is an eternal optimist still dream...,Alice Scott and Hayden Anderson are both on Li...
